## GET AND CLEAN DATA FROM DATA FOLDER BEFORE RUNNING THIS SCRIPT!!!

#### I ran this code using Google Colab since the training takes awhile, but you're welcome to use any platform you like!

In [ ]:
# Unzip the cleaned dataset
!unzip "/content/asl_data.zip"

In [ ]:
# Define variables for you models and dataset
DATA_DIR = "/content/asl_data"         # root containing subfolders per class
IMG_SIZE = (150, 150)
BATCH = 64                    # try 64 first; drop to 32 if memory is tight
EPOCHS = 20
VAL_SPLIT = 0.2               # train/val split from the directory
SEED = 42

In [ ]:
# Import necessary packages and tools
import os, sys, itertools, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Check for bad files in the dataset
import os

bad_files = []

for root, dirs, files in os.walk(DATA_DIR):
    for f in files:
        path = os.path.join(root, f)
        # Check empty file
        if os.path.getsize(path) == 0:
            bad_files.append(path)

print("Bad files:", bad_files)
print("Count:", len(bad_files))


In [ ]:
# Remove bad files from the dataset
import os

for path in bad_files:
    try:
        os.remove(path)
        print("Deleted:", path)
    except Exception as e:
        print("Error deleting", path, e)


In [ ]:
# Split dataset into training and validation sets

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
)

# Get class names and number of classes (if correctly loaded)
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)


In [ ]:
# Setting up the model with Transfer Learning
from tensorflow import keras
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras import layers, models, regularizers, optimizers
import tensorflow as tf
import matplotlib.pyplot as plt

# Use a Pre-trained Network with Transfer Learning
# Define input shape required by ResNet50V2
input_shape = (224, 224, 3)

# Create base model with pre-trained weights
base_model = ResNet50V2(weights='imagenet',
                        include_top=False,  # exclude the top layers since we're adding our own
                        input_shape=input_shape)

# Build the full model with custom top layers
# Youre free to modify the architecture as needed but this is a good starting point!
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0005)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(26, activation='softmax')
])

# Resize and preprocess datasets
def preprocess_dataset(ds):
    ds = ds.map(lambda x, y: (tf.image.resize(x, [224, 224]), y))
    ds = ds.map(lambda x, y: (preprocess_input(x), y))
    return ds

train_ds = preprocess_dataset(train_ds)
validation_ds = preprocess_dataset(val_ds)

# Setup callbacks to help reduce plateu and save best model!
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1
)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=8, restore_best_weights=True, verbose=1
)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_resnet_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

model.summary()

In [ ]:
# Phase 1 code here
# Freeze the base model and train only the top layers first
print("Phase 1: Training only the top layers with frozen base model...")

base_model.trainable = False

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_phase1 = model.fit(
    train_ds,
    epochs=15, # You can adjust the number of epochs as needed
    validation_data=validation_ds,
    callbacks=[reduce_lr, early_stop, checkpoint] # Remember to include your callbacks here!
)

In [ ]:
# Phase 2 code here
print("\nPhase 2: Fine-tuning the model with small learning rate...")

# Unfreeze the base model, but keep BatchNorm layers frozen
base_model.trainable = True
for layer in base_model.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

# Compile model for Phase 2 with much smaller learning rate as we approach final convergence
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train Phase 2
history_phase2 = model.fit(
    train_ds,
    epochs=15,
    validation_data=validation_ds,
    callbacks=[reduce_lr, early_stop, checkpoint]
)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# Load the best model and evaluate
best_model = tf.keras.models.load_model('best_resnet_model.keras')
evaluation = best_model.evaluate(validation_ds)
print(f"Final validation accuracy: {evaluation[1]:.4f}")

# Get true and predicted labels
y_true = []
y_pred = []

for images, labels in validation_ds:
    preds = best_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

# Display Confusion Matrix to see detailed performance
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(xticks_rotation=90)
plt.title("Confusion Matrix - Validation Set")
plt.show()